<a href="https://colab.research.google.com/github/simulate111/Textual-Data-Analysis-25/blob/main/ex14_solved.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# The solution steps

1. Load the data as usual
2. Define a DSPy signature which describes the task as mapping a document (str) into a list of person,org,role tuples;
3. Run the program
4. Inspect the prompt used through e.g. llm.history[-1]

In [ ]:
!pip -q install dspy

In [ ]:
import dspy
import unicodedata
import re
import json
import random

#api_keys.py has a GPT4o_API_KEY variable
#from api_keys import *

GPT4o_API_KEY = "sk-xxxxxxxxxx"#'your_api_key_here'

lm = dspy.LM('openai/gpt-4o-mini', api_key=GPT4o_API_KEY)
dspy.configure(lm=lm)

In [ ]:
!wget http://dl.turkunlp.org/TKO_8964_2023/news-en-2021.jsonl

--2025-03-12 08:27:54--  http://dl.turkunlp.org/TKO_8964_2023/news-en-2021.jsonl
Resolving dl.turkunlp.org (dl.turkunlp.org)... 195.148.30.23
Connecting to dl.turkunlp.org (dl.turkunlp.org)|195.148.30.23|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3385882 (3.2M) [application/octet-stream]
Saving to: ‘news-en-2021.jsonl’

news-en-2021.jsonl  100%[===================>]   3.23M  2.28MB/s    in 1.4s    

2025-03-12 08:27:56 (2.28 MB/s) - ‘news-en-2021.jsonl’ saved [3385882/3385882]



In [ ]:
news=[]
with open("news-en-2021.jsonl") as f:
    for line in f:
        d=json.loads(line)
        news.append(d)
news_dspy=[dspy.Example(document=s["text"]).with_inputs("document") for s in news]
print(news_dspy[0])


Example({'document': 'Finland\'s government is pushing ahead with plans to introduce a Covid pass, following a meeting of ministers at the House of the Estates in Helsinki on Thursday afternoon. \n "There are still many open questions that need to be answered. At this point, it is impossible to promise that the pass will come or when it will come," Prime Minister  Sanna Marin  (SDP) told the media following the conclusion of the meeting. \n "The government has given the green light to the Covid pass and preparations will continue," Marin added. \n Minister of Economic Affairs  Mika Lintilä  (Cen) told reporters immediately after the meeting that there was broad agreement between the coalition parties over the need for the certificate. \n "It [the pass] is an important tool so that we will not need restrictions any more," Lintilä said. \n The government also decided at Thursday afternoon\'s meeting to offer coronavirus vaccines to all 12- to 15-year-olds, starting as early as next week.

# DSPy Signature

* I opt to define it as a class which allows me to add the hint strings

In [ ]:
class Relation(dspy.Signature):
    """Describes a relation between a person and an organization, most likely employment, or being a representative, or similar."""

    document: str = dspy.InputField(desc="Input news document")
    person_organization_relationship: list[tuple[str,str,str]] = dspy.OutputField(desc="extracted triples of person,organization,nature-of-relationship")

compare_prog=dspy.ChainOfThought(signature=Relation)


In [ ]:
import tqdm
responses=[]

for item in tqdm.tqdm(news_dspy[:30]):
    response=compare_prog(**item)
    responses.append(response)


100%|██████████| 30/30 [02:32<00:00,  5.09s/it]


In [ ]:
print(responses[0])

Prediction(
    reasoning='The document discusses the actions and statements of Finnish government officials regarding the introduction of a Covid pass and related measures. It highlights the roles of Prime Minister Sanna Marin and Minister of Economic Affairs Mika Lintilä, indicating their positions within the Finnish government. Their statements reflect their responsibilities and involvement in government decisions, establishing a clear relationship between these individuals and the organization they represent, which is the Finnish government.',
    person_organization_relationship=[('Sanna Marin', 'Finnish government', 'Prime Minister'), ('Mika Lintilä', 'Finnish government', 'Minister of Economic Affairs')]
)


# Save the answers

* It might be wise to save the responses so that we do not need to rerun the the program
* For that, it is necessary to turn the responses into simpler objects

In [ ]:

responses_py=[{"reasoning":r.reasoning,"triples":r.person_organization_relationship} for r in responses]

with open("responses.jsonl","wt") as f:
    for r in responses_py:
        print(json.dumps(r),file=f)

In [ ]:
lm.history[-1]

{'prompt': None,
 'messages': [{'role': 'system',
   'content': 'Your input fields are:\n1. `document` (str): Input news document\n\nYour output fields are:\n1. `reasoning` (str)\n2. `person_organization_relationship` (list[tuple[str, str, str]]): extracted triples of person,organization,nature-of-relationship\n\nAll interactions will be structured in the following way, with the appropriate values filled in.\n\nInputs will have the following structure:\n\n[[ ## document ## ]]\n{document}\n\nOutputs will be a JSON object with the following fields.\n\n{\n  "reasoning": "{reasoning}",\n  "person_organization_relationship": "{person_organization_relationship}        # note: the value you produce must adhere to the JSON schema: {\\"items\\": {\\"maxItems\\": 3, \\"minItems\\": 3, \\"prefixItems\\": [{\\"type\\": \\"string\\"}, {\\"type\\": \\"string\\"}, {\\"type\\": \\"string\\"}], \\"type\\": \\"array\\"}, \\"type\\": \\"array\\"}"\n}\n\nIn adhering to this structure, your objective is: \

In [ ]:
for r in responses_py:
    for p,o,r in r["triples"]:
        print(f"Person: {p}    Org: {o}     Rel: {r}")

Person: Sanna Marin    Org: Finnish government     Rel: Prime Minister
Person: Mika Lintilä    Org: Finnish government     Rel: Minister of Economic Affairs
Person: Timo Metsola    Org: Vuokraturva     Rel: board chair
Person: Emma Terho    Org: International Olympic Committee Athletes’ Commission     Rel: Chair
Person: Kirsty Coventry    Org: International Olympic Committee Athletes’ Commission     Rel: Former Chair
Person: Peter Tallberg    Org: International Olympic Committee Athletes’ Commission     Rel: Former Member
Person: Niina Kauppinen    Org: Helsinki and Uusimaa Hospital District     Rel: Communications Manager
Person: Kari Kristeri    Org: Kymenlaakso Hospital District     Rel: Chief Administrative Officer
Person: Risto Pietikäinen    Org: Kymenlaakso Hospital District     Rel: Chief Physician
Person: Markku Broas    Org: Lapland's Hospital District     Rel: Chief Physician
Person: Veikko Karvanen    Org: South-Savo Hospital District     Rel: Chief Physician
Person: Anne K

# Observations

* By inspecting the output, we only evaluate *precision* and not *recall* - be mindful of that
* The model took a pretty "broad" view of what relation can mean
* The model took an excessive freedom in assigning *person* to entities which are not a human
